<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import</a></span><ul class="toc-item"><li><span><a href="#email_preprocess.py" data-toc-modified-id="email_preprocess.py-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>email_preprocess.py</a></span></li></ul></li></ul></div>

# Email_Naive Bayes

## Import 

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import requests
import pickle 

import scipy as sp
import nltk

import joblib

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif

from pathlib import Path
import os

os.getcwd()

'/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/Email_NB'

In [2]:
path = Path('/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/Email_NB/Data')

### email_preprocess.py

In [37]:
def preprocess(words_file = '/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/Email_NB/ud120-projects/tools/word_data.pkl', authors_file = '/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/Email_NB/ud120-projects/tools/email_authors.pkl'):
    """
        This function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) 
        and performs a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features
        
        after this, the fueatres nad labels are put into numpy arrays, which play nice
        with sklearn functions
        
        4 objects are returned:
            -- training/testing features
            -- training/testing labels
    """
    
    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the test learning mini-project
    authors_file_handler = open(authors_file, 'rb')
    authors = joblib.load(authors_file)
    
    
    words_file_handler = open(words_file, 'rb')
    word_data = joblib.load(words_file_handler)
    
    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = train_test_split(word_data, authors, test_size=0.1, random_state=42)
    
    
    ### text vectorization -- go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed = vectorizer.transform(features_test)
    
    
    ### feature selection, because text is super high dimensional and
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed = selector.transform(features_test_transformed).toarray()
    
    
    ### info on the data
    print("No. of Chris training emails : ", sum(labels_train))
    print("No. of Sara training emails : ", sum(labels_train)-sum(labels_train))
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test

In [38]:
features_train, features_test, labels_train, labels_test = preprocess()

No. of Chris training emails :  7936
No. of Sara training emails :  0


In [43]:
#!/usr/bin/python3

""" 
    This is the code to accompany the Lesson 1 (Naive Bayes) mini-project. 
    Use a Naive Bayes Classifier to identify emails by their authors
    
    authors and labels:
    Sara has label 0
    Chris has label 1
"""
    
#import sys
from time import time
#sys.path.append("../tools/")
#from email_preprocess import preprocess


### Split train and test dataset
X_train, X_test, y_train, y_test = preprocess()


from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
t0 = time()
clf.fit(X_train, y_train)
print("training time: ", round(time()-t0, 3), "s")

t1 = time()
pred = clf.predict(X_test)
print("predicting time: ", round(time()-t1, 3), "s")

from sklearn.metrics import accuracy_score
t2 = time()
print("accuracy:", accuracy_score(pred, y_test))
print("scoring time: ", round(time()-t2, 3), "s")



No. of Chris training emails :  7936
No. of Sara training emails :  0
training time:  0.751 s
predicting time:  0.035 s
accuracy: 0.9732650739476678
scoring time:  0.001 s
